In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
sns.set(style="white", context='poster')
import math
from scipy.integrate import solve_ivp

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
#from model_equations_separate_NC import *
from model_equations_separate_NC_store_numba import *


In [ ]:
morder = ['MIN', 'OVERFLOW', 'MIXOTROPH', 'EXOENZYME', 'ROS',]
media_order = ['lowN', 'pro99']
mpalette = 'twilight'
mpalette = sns.color_palette(mpalette, n_colors=5)

morder= [morder[1]] + morder[3:]
mpalette= [mpalette[1]] + mpalette[3:]
sns.color_palette(mpalette)


In [ ]:
gorder = [ 'Strong', 'Sustained', 'Weak', 'Inhibited','Axenic', 'Other']
gpalette = sns.color_palette('Set2', n_colors=4).as_hex() + ['black', 'lightgrey']
sns.color_palette(gpalette)

In [ ]:

HCOLOR = '#ffc58f'
HCOLOR_light = '#ffd2a9'
PCOLOR = '#99cc99'
PCOLOR_light = '#aad5aa'

DONCOLOR = '#7285b7'
RDONCOLOR = 'black' #'#00334D'
DINCOLOR = '#bbdaff' #'#000066'

DOCCOLOR = '#cc6666'
RDOCCOLOR = 'black' #'#00334D'
DICCOLOR = '#ff9da4' #'#000066'

SHCOLOR = '#ffa98f'
SPCOLOR = '#326232'


ccnpalette = [PCOLOR, PCOLOR_light, HCOLOR, HCOLOR_light, SPCOLOR, SHCOLOR,DONCOLOR, RDONCOLOR, DINCOLOR]
ccnorder = ['Bp', 'Np', 'Bh','Nh',  'ABp', 'ABh', 'DON', 'RDON', 'DIN', ]
cccpalette = [PCOLOR, PCOLOR_light, HCOLOR, HCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
cccorder = ['Bp[C]', 'Cp', 'Bh[C]', 'Ch', 'ABp[C]', 'ABh[C]', 'DOC', 'RDOC', 'DIC']
ccxpalette = [PCOLOR, HCOLOR]
ccxorder = ['Xp', 'Xh']
hnpalette = [ HCOLOR_light, SPCOLOR, SHCOLOR, DONCOLOR, RDONCOLOR, DINCOLOR]
hnorder = [ 'Bh', 'ABp', 'ABh', 'DON', 'RDON', 'DIN']
hcpalette = [ HCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
hcorder = [ 'Bh[C]', 'ABp[C]', 'ABh[C]', 'DOC', 'RDOC', 'DIC']
pnpalette = [ PCOLOR_light, SPCOLOR, SHCOLOR, DONCOLOR, RDONCOLOR, DINCOLOR]
pnorder = [ 'Bp', 'ABp', 'ABh', 'DON', 'RDON', 'DIN']
pcpalette = [ PCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
pcorder = [ 'Bp[C]', 'ABp[C]', 'ABh[C]', 'DOC', 'RDOC', 'DIC']
sns.color_palette(ccnpalette + cccpalette, )
# 


In [ ]:
plt.rcParams['figure.dpi']= 300

In [ ]:

 
sns.color_palette([PCOLOR, HCOLOR, PCOLOR_light, HCOLOR_light] )


In [ ]:
refdf = pd.read_excel('reference_10cc_axenic.xlsx')
refp99df = pd.read_excel('reference_pro99_axenic.xlsx')


In [ ]:
dpath = '/fast_data/Osnat/RECYCLE_MODEL/results/shgo2/out'
dpath = 'results'

In [ ]:
os.listdir(dpath)

In [ ]:
pro99_mode = False 
which_organism = 'ponly'
organism_to_tune = 'PRO'
(var_names, init_var_vals, intermediate_names, calc_dydt, prepare_params_tuple
) = get_constants_per_organism(pro99_mode, which_organism)
for model in ['OVERFLOW', 'MIXOTROPH', 'ROS', 'MIN', 'EXOENZYME']:
    params_to_update, bounds, log_params = get_param_tuning_values(model, organism_to_tune)
    


In [ ]:
dpath = 'results'
sum_df = pd.read_csv(os.path.join(dpath,'monte_add_het_clean_sum.csv.gz',))
minmse_df = pd.read_csv('ML_model/monte_add_het_predicted_classes.csv.gz')


In [ ]:
strong_and_sustained_vpros = set(minmse_df.loc[minmse_df.y_pred.isin(['Strong', 'Sustained']), 'VPRO'])
len(strong_and_sustained_vpros)

In [ ]:
vdf1 = minmse_df.loc[~minmse_df.y_pred.isin(['Other'])].groupby(['model', 'VPRO']).y_pred.agg(lambda x : np.array2string(np.sort(x.unique()))).reset_index()
vdf1

In [ ]:
vdf1['y_pred'] = vdf1['y_pred'].str.replace("' '", ',', regex=False)
vdf1['y_pred'] = vdf1['y_pred'].str.replace("['", '', regex=False)
vdf1['y_pred'] = vdf1['y_pred'].str.replace("']", '', regex=False)

In [ ]:
vdf1

In [ ]:
vdf1.groupby(['model', 'y_pred']).size().reset_index().pivot(index='y_pred', columns='model').fillna(0)

In [ ]:
sum_df = pd.merge(minmse_df, 
                  sum_df, 
                  on='run_id', how='left')

In [ ]:
sum_df

### vmax per cell

uMN_per_cell = Qp = 12.5 fg/cell * 1e-9 fg->uM  / 14 (N MW)

vmax_per_sec_per_uMN = uM sec -1 uM N -1

vmax_per_day_per_uMN = vmax_per_sec_per_uMN * seconds_in_day

vmax_per_day_per_cell = vmax_per_day_per_uMN * Qp




In [ ]:
sum_df.columns

In [ ]:
df = sum_df.copy()

In [ ]:
per_sec_params = [
    'VmaxICp', 'VmaxICh', 'VmaxINp',
    'VmaxINh', 'VmaxOCp', 'VmaxOCh', 'VmaxONp', 'VmaxONh', 
    #'KICp', 'KICh', 'KINp', 'KINh', 'KOCp', 'KOCh', 'KONp', 'KONh', 
    'Mp', 'Mh', #'gammaDp', 'gammaDh', 'Rp', 'Rh', 'QCmaxp', 'QCminp', 'QCmaxh', 'QCminh', 
    'KdecayDON', 'KprodEXOp',
    'KprodEXOh',  'Koverflowp', 'Koverflowh', 
    'KprodROSp', 'KprodROSh', 'KlossROSp', 'KlossROSh', 
    'omegaP', 'omegaH', 
    ]
for p in per_sec_params:
    df[p] = df[p] * seconds_in_day

In [ ]:
sum_df.columns

In [ ]:
#additional_variables = [ 'Vmax/K ICp', 'Vmax/K INp', 'R* INp', 'R* ICp', 'Vmax/Vmax Ip']
additional_variables = [ ]
id_vars= ['run_id', 'y_pred', 'max_prob', 'idx',  'model', 'VPRO',]
def _get_params_df(model):
    pparams_to_update, bounds, log_params = get_param_tuning_values(model, 'PRO')
    hparams_to_update, bounds, log_params = get_param_tuning_values(model, 'HET')
    param_vals_map = get_param_vals(model)
    for p in per_sec_params:
        if p in param_vals_map:
            param_vals_map[p] = param_vals_map[p] * seconds_in_day    


    param_vals_df = df.loc[df.model.isin([model]), 
                                id_vars + pparams_to_update  + hparams_to_update + additional_variables]
    mparam_vals = param_vals_df.melt( id_vars=id_vars)    
    mparam_vals['default'] = mparam_vals['variable'].map(param_vals_map)
    mparam_vals['FC'] = np.log2(mparam_vals.value / mparam_vals.default)
    return mparam_vals
mparams_df = pd.concat([_get_params_df(model) for model in morder], ignore_index=True)


In [ ]:
mparams_df.variable.unique()

In [ ]:
mparams_df.loc[mparams_df.variable.isna()]

In [ ]:
sns.set_context('poster')
g = sns.catplot(
    data=mparams_df.loc[~mparams_df['FC'].isna()].reset_index(), 
    col='variable', x='FC', y='model', col_wrap=6, order=morder,
    hue='model', hue_order=morder, palette=mpalette, 
    legend=False,
).set_titles(col_template='{col_name}', row_template='{row_name}').set(xlabel='log2FC')

for (#row_val,
     col_val), ax in g.axes_dict.items():
    #refmean = refmaxday_df.loc[refmaxday_df['media'].isin([col_val]), col].mean()
    for i in [-3,-2,-1,0,1,2,3]:
        ax.axvline(i, color='red', ls='--', alpha=0.5, lw=0.5)


In [ ]:
sns.set_context('poster')
for m in morder:
    g = sns.catplot(
        data=mparams_df.loc[~mparams_df['FC'].isna() & mparams_df.model.isin([m])].reset_index(), 
        col='variable', x='FC', y='y_pred', col_wrap=6, order=gorder,
        hue='y_pred', hue_order=gorder, palette=gpalette, 
        #dodge=True,
        legend=False,
    ).set_titles(col_template='{col_name}', row_template='{row_name}').set(xlabel='log2FC')

    for (#row_val,
         col_val), ax in g.axes_dict.items():
        #refmean = refmaxday_df.loc[refmaxday_df['media'].isin([col_val]), col].mean()
        for i in [-3,-2,-1,0,1,2,3]:
            ax.axvline(i, color='red', ls='--', alpha=0.5, lw=0.5)
    plt.suptitle(m, y=1.02)

In [ ]:
mparams_df.variable.unique()

In [ ]:
from scipy.stats import pearsonr

def corrfunc(x, y, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    r, _ = pearsonr(x, y)
    ax = ax or plt.gca()
    ax.annotate(f'ρ = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes)

In [ ]:
mparams_df.variable.nunique()

In [ ]:
def _get_corrs(model, y_pred):
    cur_pparams_to_update, _, _ = get_param_tuning_values(model, 'PRO')
    cur_hparams_to_update, _, _ = get_param_tuning_values(model, 'HET')
    cur_params_to_update = cur_pparams_to_update + cur_hparams_to_update

    tmp = mparams_df.loc[
                    mparams_df.model.isin([model]) &
                    mparams_df.y_pred.isin([y_pred]) 
                ]

    mtmp = tmp.pivot_table(index='idx', columns='variable', values='FC')[cur_params_to_update]
    corr_df = mtmp.corr()
    trimask = np.triu(np.ones_like(corr_df, dtype=bool))    
    corr_unstack = corr_df.mask(trimask).unstack().dropna()
    res_df = corr_unstack.rename_axis(['var1', 'var2']).reset_index().rename(columns={0:'correlation'})
    res_df['model'] = model
    res_df['y_pred'] = y_pred
    return res_df
corr_df = pd.concat([_get_corrs(m, y) for m in morder for y in gorder], ignore_index=True)
corr_df

In [ ]:
corr_df['pair'] = corr_df.var1 +' / ' +  corr_df.var2

In [ ]:
df.groupby(['y_pred','model'],).size()

In [ ]:
sns.set_context('paper')
for m in morder:
    corr_df_filtered = corr_df.loc[(corr_df.correlation > 0.3) | (corr_df.correlation < -0.3)]
    corr_df_filtered = corr_df_filtered.loc[corr_df_filtered.model.isin([m])]
    if m == 'ROS':
        corr_df_filtered = corr_df_filtered.loc[~corr_df_filtered.y_pred.isin(['Inhibited'])]
        
    
    pcorr_df = corr_df.loc[
        corr_df.pair.isin(corr_df_filtered.pair) &
        corr_df.model.isin([m]) 
        
    ].pivot_table(index='pair', columns=['model', 'y_pred' ], 
                                   values='correlation', fill_value=3, dropna=True)
    pcorr_df = pcorr_df.reindex(columns=pcorr_df.columns.reindex(morder, level=0)[0])

    g = sns.clustermap(
        pcorr_df, 
        cmap='coolwarm', 
        mask=pcorr_df==3,
        col_cluster=False,
        annot=True, fmt='.2f',
        figsize=(15,15),
        vmax=1, vmin=-1,
    )
    ax = g.ax_heatmap  
    plt.suptitle(m, y=1.02)
    #for i in [2,4,6,8]:
    #    ax.axvline(i, c='black')


In [ ]:
corr_df

In [ ]:
corr_df.loc[
    (~corr_df.y_pred.isin(['Inhibited'])) &
    (np.abs(corr_df.correlation) > 0.5)
] .groupby(['model', 'pair']).size().reset_index().pivot(index='pair', columns='model')#.head(50)

In [ ]:
corr_df.loc[
    (corr_df.y_pred.isin(['Strong'])) &
    (np.abs(corr_df.correlation) > 0.4)
] .pivot(index='pair', columns='model', values='correlation')#.head(50)

In [ ]:
for m in morder:
    t = corr_df.loc[
        (~corr_df.y_pred.isin(['Inhibited'])) &
        corr_df.model.isin([m]) &
        (np.abs(corr_df.correlation) > 0.4),
        ['var1', 'var2']] 
    cols = list(set(t['var1']) | set(t['var2']))
    print(m, cols)
    d = df.loc[df.model.isin([m]), cols+['y_pred']]
    sns.pairplot(data=d, hue='y_pred', palette=gpalette, hue_order=gorder)
    plt.suptitle(m, y=1.01)
    

In [ ]:
KINp / Koverflowh 	NaN 	0.501060
KINp / Koverflowp 	NaN 	-0.585207
KINp / VmaxONh 	NaN 	0.621515
KOCh / KONh 	NaN 	-0.582889
Koverflowp / VmaxONh 	NaN 	-0.676780
Mp / Koverflowh 	NaN 	0.622533
Mp / Koverflowp 	NaN 	-0.542490
VmaxICp / Mp 	0.621278 	NaN
VmaxICp / VmaxINp 	-0.726773 	-0.533383
VmaxINh / VmaxONh 	NaN 	0.655003
VmaxINp / Mp 	NaN 	-0.567182

In [ ]:
pair_list = [
#('KINp' , 'VmaxONh'),
('KOCh' , 'KONh'),
('KOCh' , 'KINp'),
('KINp' , 'KINh'),
#('VmaxICp' , 'Mp'),
#('VmaxICp' , 'VmaxINp'),
#('VmaxINh' , 'VmaxONh'),
#('VmaxINp' , 'Mp'),
    ('Mp', 'Mh'),
    ('gammaDp', 'gammaDh'),
    
    
]

pair_list_ROS = [
('Mh' , 'KlossROSh'),
('Mp' , 'KlossROSh'),
('Mp' , 'omegaP'),
    
]

pair_list_OVERFLOW = [
('KINp' , 'Koverflowh'),
('KINp' , 'Koverflowp'),
('Koverflowp' , 'VmaxONh'),
('Mh' , 'Koverflowh'),
('Mp' , 'Koverflowh' ),
('Mp' , 'Koverflowp'),
]



In [ ]:
sns.set_context('poster')
for v1, v2 in pair_list:
        sns.relplot(
            data=df,
            x=v1, y=v2,
            row='model', row_order=morder, 
            col='y_pred', col_order=gorder,
            hue='y_pred', hue_order=gorder, palette=gpalette,
            facet_kws=dict(margin_titles=True)
        ).set_titles(row_template='{row_name}', col_template='{col_name}')
        plt.show()

In [ ]:
sns.set_context('poster')
rdf = df.loc[df.model.isin(['ROS'])]
for v1, v2 in pair_list_ROS:
        sns.relplot(
            data=df,
            x=v1, y=v2,
            #row='model', row_order=morder, 
            col='y_pred', col_order=gorder,
            hue='y_pred', hue_order=gorder, palette=gpalette,
            facet_kws=dict(margin_titles=True)
        ).set_titles(row_template='{row_name}', col_template='{col_name}')
        plt.show()

In [ ]:
df['gammaDh*Mh'] = df.gammaDh * df.Mh
df['gammaDp*Mp'] = df.gammaDp * df.Mp


In [ ]:
sns.set_context('poster')
for v1, v2 in [
    ('gammaDp*Mp', 'gammaDh*Mh'),
    ('gammaDp', 'Mp'),
    ('gammaDh', 'Mh'),
              ]:
        sns.relplot(
            data=df,
            x=v1, y=v2,
            row='model', row_order=morder, 
            col='y_pred', col_order=gorder,
            hue='y_pred', hue_order=gorder, palette=gpalette,
            facet_kws=dict(margin_titles=True)
        ).set_titles(row_template='{row_name}', col_template='{col_name}')
        plt.show()